In [ ]:
import numpy as np
from keras_preprocessing.image import ImageDataGenerator
from keras.api.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.api.models import Sequential
from keras.api.applications import EfficientNetV2B0  # Better than VGG16
from keras.api.optimizers import Adam
from keras.api.callbacks import EarlyStopping, ReduceLROnPlateau
from keras.api.regularizers import l2
from keras.api.layers import RandomContrast, RandomBrightness

In [ ]:

# Data Augmentation (Stronger)
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    # Geometric Augmentations
    rotation_range=50,  # Increased from 40
    width_shift_range=0.4,  # Increased from 0.2
    height_shift_range=0.4,
    shear_range=0.3,
    zoom_range=[0.7, 1.4],  # Wider range (both zoom-in and out)
    horizontal_flip=True,
    vertical_flip=True,
    # Color Augmentations
    brightness_range=[0.6, 1.5],  # More extreme lighting changes
    channel_shift_range=80.0,  # More intense color shifts (was 50)
    # Advanced Augmentations
    fill_mode='reflect',  # Better than 'nearest' for natural edges
    featurewise_center=True,  # Normalize per dataset
    featurewise_std_normalization=True,
    preprocessing_function=lambda x: RandomContrast(0.3)(RandomBrightness(0.3)(x))
)

test_datagen = ImageDataGenerator(rescale=1. / 255)

# Load Data
train_dir = r"/redd/train"
test_dir = r"/redd/test"

training_set = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical'
)

test_set = test_datagen.flow_from_directory(
    test_dir,
    target_size=(150, 150),
    batch_size=32,
    class_mode='categorical',
)

In [ ]:
# Load EfficientNetV2 (Modern Alternative to VGG16)
base_model = EfficientNetV2B0(
    weights='imagenet',
    include_top=False,
    input_shape=(150, 150, 3)
)
base_model.trainable = False  # Freeze initially

# Improved Model Architecture
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    BatchNormalization(),
    Dense(256, activation='swish', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(128, activation='swish', kernel_regularizer=l2(0.01)),
    Dropout(0.5),
    Dense(3, activation='softmax')
])

# Optimizer with LR Scheduling
optimizer = Adam(learning_rate=1e-4)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
import pickle
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6)

# Initial Training (Frozen Base)
history = model.fit(
    training_set,
    validation_data=test_set,
    epochs=5,
    callbacks=[early_stop, reduce_lr],
)
model.save("my_model.h5")
with open("history.pkl", "wb") as f:
    pickle.dump(history.history, f)

print("Model and training history saved successfully!")

In [ ]:

# Fine-Tuning (Unfreeze Top Layers)
base_model.trainable = True
for layer in base_model.layers[:-15]:  # Unfreeze last 15 layers
    layer.trainable = False

model.compile(
    optimizer=Adam(learning_rate=1e-5),  # Lower LR for fine-tuning
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:

history_fine = model.fit(
    training_set,
    validation_data=test_set,
    epochs=5,
    callbacks=[early_stop, reduce_lr],
)